In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from scipy.interpolate import make_interp_spline

In [2]:
res = pd.read_csv('data/prem/metrics.csv')
res['date'] = pd.to_datetime(res['date'])

In [4]:
res["date"].unique()

<DatetimeArray>
['2025-08-22 00:00:00', '2025-08-26 00:00:00', '2025-08-27 00:00:00',
 '2025-08-28 00:00:00', '2025-08-29 00:00:00', '2025-08-30 00:00:00',
 '2025-08-31 00:00:00', '2025-09-01 00:00:00', '2025-09-02 00:00:00',
 '2025-09-03 00:00:00', '2025-09-04 00:00:00', '2025-09-05 00:00:00',
 '2025-09-06 00:00:00', '2025-09-07 00:00:00', '2025-09-08 00:00:00',
 '2025-09-09 00:00:00', '2025-09-10 00:00:00', '2025-09-11 00:00:00',
 '2025-09-12 00:00:00', '2025-09-13 00:00:00', '2025-09-14 00:00:00',
 '2025-09-15 00:00:00', '2025-09-16 00:00:00', '2025-09-17 00:00:00',
 '2025-09-18 00:00:00', '2025-09-19 00:00:00', '2025-09-20 00:00:00',
 '2025-09-21 00:00:00', '2025-09-22 00:00:00', '2025-09-23 00:00:00',
 '2025-09-24 00:00:00', '2025-09-25 00:00:00', '2025-09-26 00:00:00',
 '2025-09-27 00:00:00', '2025-09-28 00:00:00', '2025-09-29 00:00:00',
 '2025-09-30 00:00:00', '2025-10-01 00:00:00', '2025-10-02 00:00:00',
 '2025-10-03 00:00:00', '2025-10-04 00:00:00', '2025-10-05 00:00:00',
 '20

In [20]:
change_dates = []
for i, (dt, df) in enumerate(res.groupby("date")):
    if i == 0:
        change_dates.append(dt)
    else:
        df_prev = df.copy()
        if all(df == df_prev):
            pass
        else:
            change_dates.append(dt)

In [21]:
change_dates

[Timestamp('2025-08-22 00:00:00')]

In [ ]:
from scipy.interpolate import make_interp_spline

# Create a smooth curve using cubic spline interpolation
# x_smooth = np.linspace(x.min(), x.max(), 10_000)
# spl = make_interp_spline(x, y, k=3)  # k=3 -> cubic spline
# y_smooth = spl(x_smooth)

plt.figure(figsize=(10, 6))
for name in res['name'].unique():
    subset = res[res['name'] == name]
    x = np.array([(dte - datetime(2025, 8, 22)).days + 1 for dte in subset["date"]])
    y = subset['total_diff'].values
    plt.scatter(x, y, marker='o', label=name)
    x_smooth = np.linspace(x.min(), x.max(), 10_000)
    spl = make_interp_spline(x, y, k=3)  # k=3 -> cubic spline
    y_smooth = spl(x_smooth)
    plt.plot(x_smooth, y_smooth)
plt.gca().invert_yaxis()
plt.xlabel('Date')
plt.ylabel('Rank')
plt.title('Premier League Predictions Rankings Over Time')
plt.legend()
plt.show()

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://sports.yahoo.com/nba/standings/?selectedTab=CONFERENCE"



        page = requests.get(self.url)
        soup = BeautifulSoup(page.content, "html.parser")
        tables = soup.find_all("table")

        conf_standings = []
        for table in tables[-2:]:

            df = pd.read_html(StringIO(str(table)))[0]
            conf = df.columns[0]

            cf_st = df[[conf]].rename(columns={conf: "Team"})
            cf_st["Team"] = cf_st["Team"].map(self.abbr_map)
            cf_st["Conference"] = conf.removesuffix("ern")
            cf_st["Rk"] = range(1, 16)
            cf_st["Date"] = date.today()
            conf_standings.append(
                cf_st[["Conference", "Rk", "Team", "Date"]]
            )

        standings = pd.concat(conf_standings, ignore_index=True)

        return standings.sort_values(["Conference", "Rk"]).reset_index(drop=True)
    

def get_scraper(league: str):

    league = league.lower()
    if league == "prem":
        return PremScraper()
    elif league == "nba":
        return NBAScraper()
    else:
        raise ValueError(f"Unsupported league: {league}")

In [4]:
import requests

url = "http://site.api.espn.com/apis/site/v2/sports/basketball/nba/teams"

url = "http://site.api.espn.com/apis/site/v2/sports/basketball/nba/teams/18"

data = requests.get(url).json()

In [5]:
data

{'team': {'id': '18',
  'uid': 's:40~l:46~t:18',
  'slug': 'new-york-knicks',
  'location': 'New York',
  'name': 'Knicks',
  'abbreviation': 'NY',
  'displayName': 'New York Knicks',
  'shortDisplayName': 'Knicks',
  'color': '1d428a',
  'alternateColor': 'f58426',
  'isActive': True,
  'logos': [{'href': 'https://a.espncdn.com/i/teamlogos/nba/500/ny.png',
    'width': 500,
    'height': 500,
    'alt': '',
    'rel': ['full', 'default'],
    'lastUpdated': '2024-06-25T21:17Z'},
   {'href': 'https://a.espncdn.com/i/teamlogos/nba/500-dark/ny.png',
    'width': 500,
    'height': 500,
    'alt': '',
    'rel': ['full', 'dark'],
    'lastUpdated': '2024-06-25T21:13Z'},
   {'href': 'https://a.espncdn.com/i/teamlogos/nba/500/scoreboard/ny.png',
    'width': 500,
    'height': 500,
    'alt': '',
    'rel': ['full', 'scoreboard'],
    'lastUpdated': '2024-06-25T21:18Z'},
   {'href': 'https://a.espncdn.com/i/teamlogos/nba/500-dark/scoreboard/ny.png',
    'width': 500,
    'height': 500,
    